In [1]:
import numpy as np
from steps import *
from tqdm import tqdm
from optimal import *
import matplotlib.pyplot as plt
import optuna
import pickle
import warnings
warnings.filterwarnings("error")


In [2]:
def plot_res(x_axis, results, x_label, y_label, label, title, path):
    plt.grid()
    plt.plot(x_axis, results.mean(axis=1), label=label)
    plt.fill_between(x_axis, results.mean(axis=1)+results.std(axis=1), results.mean(axis=1)-results.std(axis=1),
                     color='r', alpha=0.5)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.legend()
    plt.savefig(path+".png")
    plt.clf()

In [3]:
d = 20
r = 3
m = 20
T_r = 100
times = 10
d_sigma = 3
sigma_type = matrix_type.DIAG
s_type = matrix_type.IDENTITY

name = f"optuna_optimization_d={d}_r={r}_sigma={sigma_type}_s={s_type}_m={m}_Tr={T_r}_var={d_sigma}"
study = optuna.create_study(study_name=f'{name}',
                            storage=f'sqlite:///./../optuna/{name}.db ',
                            direction='minimize', load_if_exists=True)

[I 2023-05-09 11:48:01,420] Using an existing study with name 'optuna_optimization_d=20_r=3_sigma=matrix_type.DIAG_s=matrix_type.IDENTITY_m=20_Tr=100_var=3' instead of creating a new one.


In [4]:
# Sigma
sigma_max = 7.0
sigma_min = 0.5
spaces = 0.5
load = True

if not load:
    d_sigmas = np.linspace(sigma_min, sigma_max, int((sigma_max-sigma_min)/spaces)+1)
    results = np.zeros([len(d_sigmas), times])
    for idx, d_sigma in enumerate(d_sigmas):
        print(f" ***** Sigma={d_sigma} *****")
        regret_list, regret_mix_list = run_algorithm(d=d, r=r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=d_sigma, T_r=T_r,
                                                     **study.best_params)
        ratio = regret_list.min(axis=1) / regret_mix_list
        results[idx, :] = ratio

    pickle.dump(results, open(f'acc-ratio_vs_sigma0_{sigma_min}-{sigma_max}-{spaces}.p', 'wb'))
    pickle.dump(d_sigmas, open(f'sigma0-acc_{sigma_min}-{sigma_max}-{spaces}.p', 'wb'))

else:
    results = pickle.load(open(f'acc-ratio_vs_sigma0_{sigma_min}-{sigma_max}-{spaces}.p', 'rb'))
    d_sigmas = pickle.load(open(f'sigma0-acc_{sigma_min}-{sigma_max}-{spaces}.p', 'rb'))

plot_res(x_axis=d_sigmas, results=results,
         x_label=r'$\sigma_0$', y_label='Accuracy Ratio', label=f'd={d}, r={r}',
         title=r'results accuracy ratio vs $\sigma_0$',
         path=f"results_accuracy_ratio_sigma0_{sigma_min}-{sigma_max}-{spaces}")

Exception ignored in: <_io.FileIO name='acc-ratio_vs_sigma0_0.5-7.0-0.5.p' mode='rb' closefd=True>
Exception ignored in: <_io.FileIO name='sigma0-acc_0.5-7.0-0.5.p' mode='rb' closefd=True>


<Figure size 432x288 with 0 Axes>

In [5]:
# r
r_max = 10
r_min = 1
spaces = 1
load = False

if not load:
    d_sigma = 3
    r_list = range(r_min, r_max, spaces)
    results = np.zeros([len(r_list), times])
    for idx, r in enumerate(r_list):
        regret_list, regret_mix_list = run_algorithm(d=d, r=r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=d_sigma, T_r=T_r,
                                                     **study.best_params)
        ratio = regret_list.min(axis=1) / regret_mix_list
        results[idx, :] = ratio

    pickle.dump(results, open(f'acc-ratio_vs_r_{r_min}-{r_max}-{spaces}.p', 'wb'))
    pickle.dump(r_list, open(f'r-acc_{r_min}-{r_max}-{spaces}.p', 'wb'))

else:
    results = pickle.load(open(f'acc-ratio_vs_r_{r_min}-{r_max}-{spaces}.p', 'rb'))
    r_list = pickle.load(open(f'r-acc_{r_min}-{r_max}-{spaces}.p', 'rb'))

plot_res(x_axis=r_list, results=results,
         x_label='r', y_label='Accuracy Ratio', label=f"d={d}"+r" $\sigma_0$="+f"{d_sigma}",
         title='results accuracy ratio vs r',
         path=f"results_accuracy_ratio_r_{r_min}-{r_max}-{spaces}")


l_star=3 ; regret_mix=96.82689328684744 ; last_regret=98.23538468645073
l_star=2 ; regret_mix=32.524438142677724 ; last_regret=32.62119784711538
l_star=2 ; regret_mix=32.58299629301108 ; last_regret=32.61735970216703
l_star=2 ; regret_mix=272.06197788249256 ; last_regret=282.0381884251679
l_star=2 ; regret_mix=74.8412981994217 ; last_regret=82.88588379275443
l_star=2 ; regret_mix=74.5634186208246 ; last_regret=75.51303576238593
l_star=2 ; regret_mix=11.9681998878107 ; last_regret=12.005308138613342
l_star=2 ; regret_mix=31.321773032061728 ; last_regret=31.339689469845524
l_star=2 ; regret_mix=23.17175731420101 ; last_regret=23.188450002393445
l_star=4 ; regret_mix=18.386216339284207 ; last_regret=18.727315712464847
l_star=3 ; regret_mix=23.07454781757989 ; last_regret=23.27481819074219
l_star=3 ; regret_mix=5.441306652433337 ; last_regret=5.638095187213958
l_star=4 ; regret_mix=13.962714609639367 ; last_regret=14.255146530272066
l_star=3 ; regret_mix=8.237220070698768 ; last_regret=8.4

Exception ignored in: <_io.FileIO name='acc-ratio_vs_r_1-10-1.p' mode='wb' closefd=True>
Exception ignored in: <_io.FileIO name='r-acc_1-10-1.p' mode='wb' closefd=True>


<Figure size 432x288 with 0 Axes>

In [6]:
# d
load = False
d_max = 20
d_min = 5
spaces = 1

if not load:
    d_list = range(d_min, d_max, spaces)
    r = 3
    results = np.zeros([len(d_list), times])

    for idx, d in enumerate(d_list):
        regret_list, regret_mix_list = run_algorithm(d=d, r=r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=d_sigma, T_r=T_r,
                                                     **study.best_params)
        ratio = regret_list.min(axis=1) / regret_mix_list
        results[idx, :] = ratio

    pickle.dump(results, open(f'acc-ratio_vs_d_{d_min}-{d_max}-{spaces}.p', 'wb'))
    pickle.dump(r_list, open(f'd-acc_{d_min}-{d_max}-{spaces}.p', 'wb'))

else:
    results = pickle.load(open(f'acc-ratio_vs_d_{d_min}-{d_max}-{spaces}.p', 'rb'))
    d_list = pickle.load(open(f'd-acc_{d_min}-{d_max}-{spaces}.p', 'rb'))

plot_res(x_axis=d_list, results=results,
         x_label='d', y_label='Accuracy Ratio', label=f"r={r}"+r" $\sigma_0$="+f"{d_sigma}",
         title='results accuracy ratio vs d',
         path=f"results_accuracy_ratio_d_{d_min}-{d_max}-{spaces}")

l_star=4 ; regret_mix=0.39044804941545674 ; last_regret=0.42875717849431966
l_star=4 ; regret_mix=0.7857597123687875 ; last_regret=0.8852357583538806
l_star=4 ; regret_mix=0.022791028218224936 ; last_regret=0.024663039314516896
l_star=4 ; regret_mix=0.01001295808634206 ; last_regret=0.028819097803623163
l_star=4 ; regret_mix=1.4246359971024394 ; last_regret=1.5332598832955309
l_star=4 ; regret_mix=0.3055001278559912 ; last_regret=0.3406735783076274
l_star=4 ; regret_mix=0.5850844106131691 ; last_regret=0.6055418585321162
l_star=4 ; regret_mix=0.020574111558284303 ; last_regret=0.026071621841097004
l_star=4 ; regret_mix=0.18546217845873628 ; last_regret=0.1969612179185177
l_star=4 ; regret_mix=0.16168991637393987 ; last_regret=0.17869074992617848
l_star=4 ; regret_mix=0.2730823285325291 ; last_regret=0.2726424073939106
l_star=4 ; regret_mix=1.5684432218215376 ; last_regret=1.6327007320112004
l_star=4 ; regret_mix=0.8161102630254774 ; last_regret=0.8537706762878785
l_star=4 ; regret_mix=

Exception ignored in: <_io.FileIO name='acc-ratio_vs_d_5-20-1.p' mode='wb' closefd=True>
Exception ignored in: <_io.FileIO name='d-acc_5-20-1.p' mode='wb' closefd=True>


<Figure size 432x288 with 0 Axes>

In [7]:
# Regrets
load = False

if not load:
    d=20
    regret_list, regret_mix_list = run_algorithm(d=d, r=r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=d_sigma, T_r=T_r,
                                                     **study.best_params)
    pickle.dump(regret_list, open(f'regrets.p', 'wb'))

else:
    regret_list = pickle.load(open(f'regrets.p', 'rb'))


plot_res(x_axis=range(m+2), results=regret_list.T,
         x_label='#Iteration', y_label='Regret', label=f'd={d} r={r} '+r'$\sigma_0$='+f'{d_sigma}',
         title='Regret vs #Iteration',
         path="Regret_vs_Iteration")


l_star=5 ; regret_mix=23.4927049187163 ; last_regret=24.582149186489385
l_star=4 ; regret_mix=5.343919359564034 ; last_regret=5.924347512558396
l_star=5 ; regret_mix=4.588132819524883 ; last_regret=4.98839064916278
l_star=5 ; regret_mix=3.7525093060829935 ; last_regret=4.078394908815535
l_star=5 ; regret_mix=7.633666573148433 ; last_regret=8.126146980732745
l_star=5 ; regret_mix=36.97135746254428 ; last_regret=38.988636691770175
l_star=5 ; regret_mix=23.027321062121313 ; last_regret=23.94147033306227
l_star=4 ; regret_mix=7.421952321869519 ; last_regret=7.825119355978911
l_star=5 ; regret_mix=8.55917052613135 ; last_regret=9.239349709729952
l_star=4 ; regret_mix=2.6233740365602194 ; last_regret=2.8711952368897933


Exception ignored in: <_io.FileIO name='regrets.p' mode='wb' closefd=True>


<Figure size 432x288 with 0 Axes>